In [1]:
import sys
sys.path.append('../')
import pandas as pd
import joblib
from helper_files.modelling_functions import get_data, process_data, run_optuna_trials, evaluate_models, get_data_config, get_best_model
from helper_files.modelling_configs import *
from helper_files.etv_model import ETVModel

/Users/bradeneberhard/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '../data/processed/data_splits_1003.jblb'
train_df, test_df_time, test_df_random, test_df_thrower = get_data(data_path)
test_dfs = [test_df_random, test_df_time, test_df_thrower]
storage = 'sqlite:///model_tuning.db'
train_df = train_df[train_df['throw_distance'] >= 1]
MIRROR = True
NOISE_FACTOR=0.03
SMOTE=True

In [3]:
# features = thrower_context
# target = 'outcome' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv_base_augment_all', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [4]:
features = thrower_context + game_context
target = 'point_outcome'  
data_config = get_data_config(features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='fv_point_game_augment_all', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-10-07 13:46:12,334] A new study created in RDB with name: logreg_fv_point_game_augment_all
[I 2024-10-07 13:46:15,755] Trial 0 finished with value: 0.6005099683427593 and parameters: {}. Best is trial 0 with value: 0.6005099683427593.
[I 2024-10-07 13:46:16,541] A new study created in RDB with name: xgb_fv_point_game_augment_all
[I 2024-10-07 13:47:06,307] Trial 0 finished with value: 0.5712967450037622 and parameters: {'learning_rate': 0.00046670757021238915, 'max_depth': 9, 'n_estimators': 146}. Best is trial 0 with value: 0.5712967450037622.
[I 2024-10-07 13:47:11,934] Trial 1 finished with value: 0.5479917685787328 and parameters: {'learning_rate': 0.14320800448569002, 'max_depth': 1, 'n_estimators': 51}. Best is trial 0 with value: 0.5712967450037622.
[I 2024-10-07 13:47:32,704] Trial 2 finished with value: 0.6569764669992921 and parameters: {'learning_rate': 0.6504795697248025, 'max_depth': 7, 'n_estimators': 174}. Best is trial 2 with value: 0.6569764669992921.
[I 2024-1

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.641222,0.600599,0.597388,0.642159,0.597074,0.379897,0.637801,0.587946,0.393540,0.630402,0.576824,0.344634
1,xgb,0.843223,0.755312,0.798441,0.745195,0.750182,0.670657,0.739694,0.734070,0.657537,0.741067,0.765357,0.650722


In [5]:
# features = thrower_context + receiver_context
# target = 'completion'  
# data_config = get_data_config(features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_base_augment_all', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [6]:
features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
data_config = get_data_config(features, target, train_df, test_dfs, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_game_augment_all', delete_models=True, storage=storage)
metrics_df = evaluate_models(best_models, data_config)
metrics_df

[I 2024-10-07 13:50:33,607] A new study created in RDB with name: logreg_cp_game_augment_all
[I 2024-10-07 13:51:28,812] Trial 0 finished with value: 0.810339098642452 and parameters: {}. Best is trial 0 with value: 0.810339098642452.
[I 2024-10-07 13:51:41,833] A new study created in RDB with name: xgb_cp_game_augment_all
[I 2024-10-07 13:53:09,020] Trial 0 finished with value: 0.8084930704292652 and parameters: {'learning_rate': 0.000781901776362083, 'max_depth': 10, 'n_estimators': 116}. Best is trial 0 with value: 0.8084930704292652.
[I 2024-10-07 13:54:15,307] Trial 1 finished with value: 0.8095131802946467 and parameters: {'learning_rate': 0.002080470857638685, 'max_depth': 10, 'n_estimators': 82}. Best is trial 1 with value: 0.8095131802946467.
[I 2024-10-07 13:54:38,690] Trial 2 finished with value: 0.8398168377749977 and parameters: {'learning_rate': 0.05120140715790603, 'max_depth': 8, 'n_estimators': 34}. Best is trial 2 with value: 0.8398168377749977.
[I 2024-10-07 13:55:07

,Model,train_AUC,train_Accuracy,train_NPV,test_df_random_AUC,test_df_random_Accuracy,test_df_random_NPV,test_df_time_AUC,test_df_time_Accuracy,test_df_time_NPV,test_df_thrower_AUC,test_df_thrower_Accuracy,test_df_thrower_NPV
0,logreg,0.877816,0.810457,0.886783,0.720087,0.883189,0.275824,0.688134,0.884940,0.267275,0.743133,0.881200,0.266240
1,xgb,0.958694,0.892689,0.922956,0.803087,0.933336,0.507484,0.792577,0.934925,0.506757,0.819415,0.937092,0.508301


In [7]:
# features = thrower_context + receiver_context + throw_context + game_context + player_context
# target = 'completion' 
# data_config = get_data_config(features, target, train_df, test_dfs, False)
# best_models, best_params = run_optuna_trials(data_config, model_config, suffix='cp_game_player', delete_models=True, storage=storage)
# metrics_df = evaluate_models(best_models, data_config)
# metrics_df

In [8]:
study_name='xgb_cp_game_augment_all'
features = thrower_context + receiver_context + throw_context + game_context
target = 'completion'
cp_model = get_best_model(study_name, storage, train_df, test_dfs, model_config, features, target, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)
study_name='xgb_fv_point_game_augment_all'
features = thrower_context + game_context
target = 'point_outcome'
fv_model= get_best_model(study_name, storage, train_df, test_dfs, model_config, features, target, mirror=MIRROR, noise_factor=NOISE_FACTOR, smote=SMOTE)


etv_model = ETVModel(cp_model, fv_model)
joblib.dump({'cp_model':cp_model, 'fv_model':fv_model, 'etv_model':etv_model}, '../data/models/all_models_augment_all.jblb')

['../data/models/all_models_augment_all.jblb']